# Tutorial 1: Generate hard-links for consecutive ST slices

### import necessary dependencies

In [1]:
from datasets.st_loading_utils import load_DLPFC, create_dictionary_mnn, load_mHypothalamus
from datasets.data_proc import Cal_Spatial_Net
import scipy
import os
import pickle

from utils import (
    build_args_ST,
    create_optimizer,
    set_random_seed
)

from localMG_main import localMG

### HP setup

In [2]:
args = build_args_ST()
args.max_epoch=2000
args.max_epoch_triplet=500
args.section_ids="151507,151508"
args.num_class=7
args.num_hidden="512,32"
args.alpha_l=1
args.lam=1 
args.loss_fn="sce" 
args.mask_rate=0.5 
args.in_drop=0 
args.attn_drop=0 
args.remask_rate=0.1
args.seeds=[2023] 
args.num_remasking=1 
args.hvgs=3000 
args.dataset="DLPFC" 
args.consecutive_prior=1 
args.lr=0.001

#### remember to change this to your data path
args.st_data_dir="path/to/stdata"


In [3]:
args

Namespace(seeds=[2023], dataset='DLPFC', exp_fig_dir='./', h5ad_save_dir='./', st_data_dir='/home/yunfei/spatial_benchmarking/benchmarking_data/DLPFC12', pi_dir='./', consecutive_prior=1, section_ids='151507,151508', num_class=7, hvgs=3000, device=3, max_epoch=2000, max_epoch_triplet=500, warmup_steps=-1, num_heads=1, num_out_heads=1, num_layers=2, num_dec_layers=2, num_remasking=1, num_hidden='512,32', residual=False, in_drop=0, attn_drop=0, norm=None, lr=0.001, weight_decay=0, negative_slope=0.2, activation='elu', mask_rate=0.5, remask_rate=0.1, remask_method='random', mask_type='mask', mask_method='random', drop_edge_rate=0.0, encoder='gat', decoder='gat', loss_fn='sce', alpha_l=1, optimizer='adam', linear_prob=False, no_pretrain=False, checkpoint_path=None, use_cfg=False, logging=False, scheduler=False, batch_size=512, sampling_method='saint', label_rate=1.0, lam=1, delayed_ema_epoch=0, replace_rate=0.0, momentum=0.996, load_model=False)

### Generate Hard-links

In [ ]:
pi, Batch_list = localMG(args)

### save/load Hard-links

In [ ]:
S = scipy.sparse.csr_matrix(pi)
file = open(os.path.join(args.save_pi_dir, "pi_151507_151508.pickle"),'wb')
pickle.dump(S, file)

"""
file = open("pi_151507_151508.pickle",'rb') 
S = pickle.load(file)
S.toarray()

to retrieve from saved pi file
"""